In [1]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import pickle
import random
import numpy as np
import glob
from collections import defaultdict, Counter
from Bio import SeqIO

In [2]:
fasta_dir = 'C:\\Users\\joewa\\Work\\git\\amr-data\\fasta\\*.fna'
fasta_dir = 'C:\\Users\\Vinny\\work\\amr-data\\alex_data\\fasta\\*.fna'
sequences = {}

for fastafile in glob.glob(fasta_dir):
    with open(fastafile, "r") as handle:
        for record in SeqIO.parse(handle, "fasta"):
                sequences[record.id]= record.seq
                print(record.id, len(record.seq))

550.2144.con.0001 1113344
550.2144.con.0002 637658
550.2144.con.0003 499440
550.2144.con.0004 476566
550.2144.con.0005 413516
550.2144.con.0006 329368
550.2144.con.0007 307359
550.2144.con.0008 202609
550.2144.con.0009 150038
550.2144.con.0010 121429
550.2144.con.0011 115352
550.2144.con.0012 112677
550.2144.con.0013 74955
550.2144.con.0014 47820
550.2144.con.0015 39536
550.2144.con.0016 29047
550.2144.con.0017 26348
550.2144.con.0018 24984
550.2144.con.0019 24847
550.2144.con.0020 17910
550.2144.con.0021 9709
550.2144.con.0022 7456
550.2144.con.0023 5513
550.2144.con.0024 4982
550.2144.con.0025 4894
550.2144.con.0026 4020
550.2144.con.0027 2801
550.2144.con.0028 2622
550.2144.con.0029 1956
550.2144.con.0030 1651
550.2144.con.0031 1537
550.2144.con.0032 1357
550.2144.con.0033 1228
550.2144.con.0034 1221
550.2144.con.0035 982
550.2144.con.0036 820
550.2144.con.0037 674
550.2144.con.0038 618
550.2144.con.0039 597
550.2144.con.0040 404
550.2144.con.0041 321
550.2144.con.0042 294
550.2144.

550.2159.con.0003 516763
550.2159.con.0004 500638
550.2159.con.0005 449801
550.2159.con.0006 341604
550.2159.con.0007 215296
550.2159.con.0008 193557
550.2159.con.0009 182575
550.2159.con.0010 143933
550.2159.con.0011 124012
550.2159.con.0012 122558
550.2159.con.0013 113121
550.2159.con.0014 104414
550.2159.con.0015 104124
550.2159.con.0016 90833
550.2159.con.0017 63999
550.2159.con.0018 41663
550.2159.con.0019 29137
550.2159.con.0020 26250
550.2159.con.0021 26243
550.2159.con.0022 14516
550.2159.con.0023 11738
550.2159.con.0024 10346
550.2159.con.0025 8772
550.2159.con.0026 7518
550.2159.con.0027 5513
550.2159.con.0028 5442
550.2159.con.0029 4983
550.2159.con.0030 3944
550.2159.con.0031 2622
550.2159.con.0032 1659
550.2159.con.0033 1336
550.2159.con.0034 1293
550.2159.con.0035 1085
550.2159.con.0036 1000
550.2159.con.0037 890
550.2159.con.0038 814
550.2159.con.0039 736
550.2159.con.0040 657
550.2159.con.0041 632
550.2159.con.0042 626
550.2159.con.0043 594
550.2159.con.0044 566
550.215

In [3]:
first = list(sequences.keys())[0]

In [4]:
sequences[first]

Seq('aacgttgaagacgacgacgttgataggtcgggtgtgtaagcgcagcgatgcgtt...aga', SingleLetterAlphabet())

In [5]:
len(list(sequences.keys()))

561

In [7]:
counters = []
for seq_id in range(len(list(sequences.keys()))):    
    seq = sequences[list(sequences.keys())[seq_id]]
    k = 5
    kmers = []
    for i in range(len(seq)-k):
        kmers.append(seq[i:i+k])
    cnt = Counter()
    for kmer in kmers:
        cnt[kmer] += 1
    counters.append(cnt)
    print(seq_id)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [22]:
all_keys = set()
for i in range(len(counters)):
    all_keys.update(counters[i].keys())
list(all_keys)

[Seq('agatt', SingleLetterAlphabet()),
 Seq('tgggc', SingleLetterAlphabet()),
 Seq('gcgag', SingleLetterAlphabet()),
 Seq('ggaag', SingleLetterAlphabet()),
 Seq('ttggt', SingleLetterAlphabet()),
 Seq('aacac', SingleLetterAlphabet()),
 Seq('gtcaa', SingleLetterAlphabet()),
 Seq('atgtc', SingleLetterAlphabet()),
 Seq('actcc', SingleLetterAlphabet()),
 Seq('cttaa', SingleLetterAlphabet()),
 Seq('gtagc', SingleLetterAlphabet()),
 Seq('tggca', SingleLetterAlphabet()),
 Seq('tttac', SingleLetterAlphabet()),
 Seq('tgtgt', SingleLetterAlphabet()),
 Seq('ttatc', SingleLetterAlphabet()),
 Seq('tcttc', SingleLetterAlphabet()),
 Seq('ctact', SingleLetterAlphabet()),
 Seq('ctagc', SingleLetterAlphabet()),
 Seq('actta', SingleLetterAlphabet()),
 Seq('cgcat', SingleLetterAlphabet()),
 Seq('gtaag', SingleLetterAlphabet()),
 Seq('atgtg', SingleLetterAlphabet()),
 Seq('cttat', SingleLetterAlphabet()),
 Seq('cctct', SingleLetterAlphabet()),
 Seq('gtgtg', SingleLetterAlphabet()),
 Seq('atcca', SingleLette

In [ ]:
kmers_matrix = [[] for i in range(len(counters))]
for i in range(len(counters)):
    
    kmers_matrix[i]

In [10]:
cnt.most_common()

[(Seq('tatcg', SingleLetterAlphabet()), 3),
 (Seq('tcacg', SingleLetterAlphabet()), 2),
 (Seq('gctca', SingleLetterAlphabet()), 2),
 (Seq('aggtg', SingleLetterAlphabet()), 2),
 (Seq('ggtgt', SingleLetterAlphabet()), 2),
 (Seq('atcgc', SingleLetterAlphabet()), 2),
 (Seq('tcgct', SingleLetterAlphabet()), 2),
 (Seq('cgctg', SingleLetterAlphabet()), 2),
 (Seq('gctgt', SingleLetterAlphabet()), 2),
 (Seq('caggt', SingleLetterAlphabet()), 2),
 (Seq('tgatt', SingleLetterAlphabet()), 2),
 (Seq('gattg', SingleLetterAlphabet()), 2),
 (Seq('gaaag', SingleLetterAlphabet()), 2),
 (Seq('aaagc', SingleLetterAlphabet()), 2),
 (Seq('aagca', SingleLetterAlphabet()), 2),
 (Seq('cacgt', SingleLetterAlphabet()), 2),
 (Seq('tgtct', SingleLetterAlphabet()), 2),
 (Seq('tctcc', SingleLetterAlphabet()), 2),
 (Seq('caacg', SingleLetterAlphabet()), 2),
 (Seq('gccag', SingleLetterAlphabet()), 2),
 (Seq('ccaga', SingleLetterAlphabet()), 2),
 (Seq('attca', SingleLetterAlphabet()), 2),
 (Seq('tgtca', SingleLetterAlpha